In [1]:
import sys
import threading

import grpc
import numpy
import tensorflow as tf

from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc
from PIL import Image
import numpy as np
np.set_printoptions(suppress=True)
import cv2

In [7]:
def pretreat(inputs):
        
    crop_size = 160
    outputs = []
    for imggroup in inputs:
            
        group = []
        for image in imggroup:
                
            height = image.shape[0]
            width = image.shape[1]   
                        
            if(width>height):
                scale=float(crop_size)/float(height)
                img=np.array(cv2.resize(image,(int(width*scale+1),crop_size))).astype(np.float32)      
            else:                    
                scale=float(crop_size)/float(width)
                img=np.array(cv2.resize(image,(crop_size,int(height*scale+1)))).astype(np.float32)
                
            crop_y=int((img.shape[0]-crop_size)/2)
            crop_x=int((img.shape[1]-crop_size)/2)
            img=img[crop_y:crop_y+crop_size,crop_x:crop_x+crop_size,:]

            #print img
            
            std = np.std(img, ddof=1)
            mean = np.mean(img)
            std = np.max([std, 1.0/np.sqrt(160*160*3)])
           
            img = (img-mean)/std

            group.append(img)
            
        outputs.append(group)
    return np.array(outputs).astype(np.float32)

channel = grpc.insecure_channel('127.0.0.1:8500')
stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)

for _ in range(1):
    request = predict_pb2.PredictRequest()
    request.model_spec.name = '3daction'
    request.model_spec.signature_name = 'predict_image'
    
    dir_path = '/home/sw/data/yaochang/0/1_1537340362_89fe8072-bbd9-11e8-b3b6-6045cb7fb916/'
    inputs = []
    for i in range(16):
        file_path = dir_path + '{:0>4}'.format(i) + '.jpg'
        img = Image.open(file_path)
        inputs.append(np.array(img))
        
    dir_path = '/home/sw/data/yaochang/0/1_1537340445_bb80da5a-bbd9-11e8-b3b6-6045cb7fb916/'
    inputs2 = []
    for i in range(16):
        file_path = dir_path + '{:0>4}'.format(i) + '.jpg'
        img = Image.open(file_path)
        inputs2.append(np.array(img))
        
    image = pretreat(np.array([inputs,inputs2]))
    
    request.inputs['x'].CopyFrom(
        tf.contrib.util.make_tensor_proto(image))
    
    result_future = stub.Predict.future(request, 10)  # 5 seconds
    exception = result_future.exception()
    if exception:
        print(exception)
    else:
        response = numpy.array(
        result_future.result().outputs['logit'].float_val)
        print response.reshape([2,-1])


[[ 7.44987774 -3.7825408  -2.32236958]
 [ 7.80152798 -3.59011436 -2.53183818]]


In [6]:
def letterbox_image(image, size):
    '''resize image with unchanged aspect ratio using padding'''
    ih, iw, _ = image.shape
    w, h = size
    print type(ih),type(iw),type(w),type(h)
    scale = min(w / float(iw), h / float(ih))
    print scale
    
    nw = int(iw * scale)
    nh = int(ih * scale)

    image = cv2.resize(image, (nw, nh), interpolation=cv2.INTER_CUBIC)
    
    new_image = np.ones((416, 416, 3), dtype=np.int8) * 128
    new_image[(h - nh) // 2:(h - nh) // 2 + nh, (w - nw) // 2:(w - nw) // 2 + nw, :] = image

    return new_image


def pretreat(inputs):
    boxed_image = letterbox_image(inputs, tuple(reversed((416, 416))))
    
    image_data = np.array(boxed_image, dtype='float32')
    image_data /= 255.
    image_data = np.expand_dims(image_data, 0)  # Add batch dimension.

    return image_data

    
file_path = '/home/sw/data/yaochang/0/1_1537340445_bb80da5a-bbd9-11e8-b3b6-6045cb7fb916/0000.jpg'

img = Image.open(file_path)
img = np.array(img)

image_data = pretreat(img)

channel = grpc.insecure_channel('127.0.0.1:8500')
stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)

request = predict_pb2.PredictRequest()
request.model_spec.name = 'yolov3'
request.model_spec.signature_name = 'detect_image'
    
request.inputs['x'].CopyFrom(tf.contrib.util.make_tensor_proto(image_data))
request.inputs['shape'].CopyFrom(tf.contrib.util.make_tensor_proto([float(img.shape[0]), float(img.shape[1])]))
    
result_future = stub.Predict.future(request, 10)  # 5 seconds
exception = result_future.exception()
if exception:
    print(exception)
else:
    out_boxes = numpy.array(result_future.result().outputs['boxes'].float_val).reshape((-1,4))
    out_scores = result_future.result().outputs['scores'].float_val
    out_classes = result_future.result().outputs['classes'].int_val
    print out_boxes, out_scores, out_classes

<type 'int'> <type 'int'> <type 'int'> <type 'int'>
1.74058577406
[[157.66688538   1.95421338 234.63291931  67.25761414]
 [  1.8492595  175.17196655 213.08200073 242.93869019]
 [ 23.96910286  60.15988541 248.75239563 181.92947388]] [0.9731842279434204, 0.5309525728225708, 0.3046540319919586] [0, 0, 0]


In [5]:
file_path = '/home/sw/data/yaochang/0/1_1537340445_bb80da5a-bbd9-11e8-b3b6-6045cb7fb916/0000.jpg'
img = Image.open(file_path)
image_data = np.array(img.resize((224,224)), dtype = np.float32)/127.5 - 1.
image_data = np.expand_dims(image_data, 0)

channel = grpc.insecure_channel('127.0.0.1:8500')
stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)

request = predict_pb2.PredictRequest()
request.model_spec.name = 'mobilenet'
request.model_spec.signature_name = 'classify_image'
    
request.inputs['x'].CopyFrom(tf.contrib.util.make_tensor_proto(image_data))
    
result_future = stub.Predict.future(request, 10)  # 5 seconds
exception = result_future.exception()
if exception:
    print(exception)
else:
    res = numpy.array(result_future.result().outputs['scores'].float_val)
    print res

[0.99997985 0.00002018]
